# MaBoSS_test tutorial

The following tutorial uses p53/MDM2 model as a test case for illustrating the maboss_test framework.

## Model description:
We consider a published model of p53 response to DNA damage (PMID:19233211). p53 interacts with Mdm2, which appears in two forms, cytoplasmic and nuclear. On one hand, p53 upregulates the level of cytoplasmic Mdm2 which is then transported into the nucleus and inhibits the export of nuclear Mdm2. On the other hand, Mdm2 facilitates the degradation of p53 through ubiquitination. In the model, stress regulates the level of DNA damage, which in turn participates in the degradation process of Mdm2. p53 inhibits the DNA damage signal by promoting DNA repair.

The model can be found in the MaBoSS repository at the following link:
https://maboss.curie.fr/

# Getting started

### Installation
Install the maboss and maboss_test libraries with the following commands:
* pip/pip3 install maboss
* pip/pip3 install maboss_test

In [3]:
import maboss
import maboss_test

### Import the model

Load the model to test in the maboss format (.bnd and .cfg files)

In [4]:
p53_Mdm2_model = maboss.load("p53_Mdm2.bnd", "p53_Mdm2.cfg")   

The stable states of the model are:

In [5]:
results = p53_Mdm2_model.run()
results.get_fptable()

,FP,Proba,State,p53_b1,Mdm2nuc,p53_b2,Mdm2cyt_b1,Mdm2cyt_b2,DNAdam
0,#1,1,Mdm2nuc -- Mdm2cyt_b1,0,1,0,1,0,0


### Create the object to perform the tests
Build an instance of the MaBoSSTestCase class, linked to 'p53_Mdm2_model', with the ''detailed output'' in the following way: 

In [8]:
test_p53_Mdm2 = maboss_test.MaBoSSTestCase(p53_Mdm2_model)

Build another instance with the ''unit testing'' output putting the default argument verbose equal to False:

In [9]:
test_p53_Mdm2_mute = maboss_test.MaBoSSTestCase(p53_Mdm2_model, verbose = False)

# MaBoSSTestCase methods

### .assertStableStateProbabilityEvolution()

This function will simulate the wild type model and the model with the given mutations, both with the initial conditions I_C. It will then compare simulation results and check if the given stable state is evolving in the indicated direction. If not correct, this test will fail by raising an exception.

Parameters:	

   * mutations (dict) – The mutations to apply to the model
   * I_C – Initial conditions with which to simulate the model, it may be a list or a dictionary
   * state (dict) – The state to evaluate
   * direction (string) – The direction of the evolution of the given state, it may be: ‘increase’, ‘decrease’ or ‘stable’ 
   * digits (int): The number of digits you want to keep during the comparison



The stable states of the model are:

In [10]:
s_state = {'p53_b1':0, 'Mdm2cyt_b1':1, 'Mdm2nuc':1, 'DNAdam':0}

"Assert that: after p53 inhibition, the probability to observe s_state decreases."

In [17]:
test_p53_Mdm2.assertStableStateProbabilityEvolution({'p53_b1':'OFF'}, {}, s_state, 'decrease')

Care, you did not changed any initial condition!
True!  
The new probability of reaching the stable state is: 0.4934 
The old one is: 1


In [18]:
test_p53_Mdm2_mute.assertStableStateProbabilityEvolution({'p53_b1':'OFF'}, {}, s_state, 'decrease')

... OK


"Assert that: after Mdm2cyt_b2 activation the probability to observe s_state increases."

In [14]:
test_p53_Mdm2.assertStableStateProbabilityEvolution({'Mdm2cyt_b2':'ON'}, {}, s_state, 'increase')

Care, you did not changed any initial condition!
False!  
The new probability of reaching the stable state is: 0.6831 
The old one is: 1


In [19]:
test_p53_Mdm2_mute.assertStableStateProbabilityEvolution({'Mdm2cyt_b2':'ON'}, {}, s_state, 'increase')

False is not true


### .assertStateProbabilityEvolution()
This function will simulate the wild type model and the model with the given mutations, both with the initial conditions I_C. It will then compare simulation results and check if the given state is evolving in the indicated direction. If not correct, this test will fail by raising an exception.

(Note: the only difference with the previous function is that here the method will look for all the states and not only for the stable ones)

Parameters:	

   * mutations (dict) – The mutations to apply to the model
   * I_C – Initial conditions with which to simulate the model, it may be a list or a dictionary
   * state (dict) – The state to evaluate
   * direction (string) – The direction of the evolution of the given state, it may be: ‘increase’, ‘decrease’   or ‘stable’ 
   * int digits (int):  The number of digits you want to keep during the comparison


''Assert that: after the inhibition of Mdm2cyt the probability to observe p53_b1 active increases.''

In [20]:
test_p53_Mdm2.assertStateProbabilityEvolution({'Mdm2cyt_b1':'OFF'}, {}, {'p53_b1':1}, 'increase')

Care, you did not changed any initial condition!
True!  
The new probability of reaching the state is: 1.0 
The old one is: 0


In [52]:
test_p53_Mdm2_mute.assertStateProbabilityEvolution({'Mdm2cyt_b1':'OFF'}, {}, {'p53_b2':1}, 'increase')

... OK


''Assert that: after the inhibition of Mdm2cyt the probability to observe p53_b1 active decreases.''

In [21]:
test_p53_Mdm2.assertStateProbabilityEvolution({'Mdm2cyt_b1':'OFF'}, {}, {'p53_b1':1}, 'decrease')

Care, you did not changed any initial condition!
False!  
The new probability of reaching the state is: 1.0 
The old one is: 0


In [22]:
test_p53_Mdm2_mute.assertStateProbabilityEvolution({'Mdm2cyt_b1':'OFF'}, {}, {'p53_b1':1}, 'decrease')

False is not true


### .assertNodesDependencies()

This function will simulate the model with the given mutations. It will then extract the stable states of the model in which the activity of the nodes in condition is satisfied. It will then check that, for each selected state, the values of the nodes in nodes_expected is satisfied. If not correct, this test will fail by raising an exception.

Parameters:	

   * mutations (dict) – The mutations to apply to the model
   * condition (dict) – A list of nodes with a specifc value assigned (0:inactive or 1:active) that a state has to respect in order to be selected
   * nodes_expected (dict) – A list of nodes with a specifc value assigned (0:inactive or 1:active) that all the selected states have to respect


"Assert that in all the stable states in which Mdm2cyt_b1 is active,  Mdm2nuc is active too."

In [23]:
test_p53_Mdm2.assertNodesDependencies({}, {'Mdm2cyt_b1':1}, {'Mdm2nuc':1})

Care, you did not insert mutations!
True! 
All the states that satisfy:  {'Mdm2cyt_b1': 1}  have:  {'Mdm2nuc': 1}


In [24]:
test_p53_Mdm2_mute.assertNodesDependencies({}, {'Mdm2cyt_b1':1}, {'Mdm2nuc':1})

... OK


"Assert that in all the stable states in which Mdm2cyt_b1 is active,  p53_b1 is active too."

In [25]:
test_p53_Mdm2.assertNodesDependencies({}, {'Mdm2cyt_b1':1}, {'p53_b1':1})

Care, you did not insert mutations!
False! 
The states with  {'Mdm2cyt_b1': 1}  are : 

Probability =  1 
State:  ['Mdm2nuc', 'Mdm2cyt_b1']

Of these, those with  {'p53_b1': 1}  are: 
No one


In [26]:
test_p53_Mdm2_mute.assertNodesDependencies({}, {'Mdm2cyt_b1':1}, {'p53_b1':1})

{1: ['Mdm2nuc', 'Mdm2cyt_b1']} != None
